In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

import matplotlib.pyplot as plt
import numpy as np
import seaborn

import ising_model
import exact_ising_model as exact
import plotting
import analysis

# Metropolis
Lattice Size 4, Temperature 8.0
Energy per Site: -0.2610626220703125 +/- 0.0018771324458461633
Magnetization per Site: -1.9073486328125e-06 +/- 0.002077072882542814
m (<|M|>/N): 0.26807212829589844 +/- 0.0010193575963454566
Energy Squared: 54.25830078125 +/- 0.44047312300298547
Magnetization Squared: 28.64300537109375 +/- 0.18814095808833015
Heat Capacity per Site: 0.03599535199100501 +/- 0.00038559787097370484

# Wolff
Lattice Size 4, Temperature 8.0
Energy per Site: -0.267791748046875 +/- 0.0032271910499994372
Magnetization per Site: -0.0035552978515625 +/- 0.00286548199467902
m (<|M|>/N): 0.2700157165527344 +/- 0.0012436922244417024
Energy Squared: 55.67626953125 +/- 0.6168333494425499
Magnetization Squared: 28.962158203125 +/- 0.23760101756071275
Heat Capacity per Site: 0.036492407917194214 +/- 0.0004084473137390334

# Corboz
Energy per site : -0.263511657715  +/-  0.00150418036608
Magnetization per site : 0.000926971435547  +/-  0.00199863921378
m : 0.268798828125  +/-  0.000809964714865
Energy squared : 54.658203125  +/-  0.391778044629
Magnetization squared : 28.7371826172  +/-  0.149540865285

In [ ]:
files = ["20160513_153432_ising_wolff_10_65536_[2.169-2.369]_0.01.pickle",
        "20160513_155413_ising_wolff_20_65536_[2.169-2.369]_0.01.pickle",
        "20160513_164844_ising_wolff_40_65536_[2.169-2.369]_0.01.pickle"]
(critical_temperature,
 critical_temperature_error,
 magnetizabilities,
 magnetizations,
 heat_capacities) = analysis.data_analysis(files)

In [ ]:
analysis.find_critical_exponents(critical_temperature,
                                 critical_temperature_error,
                                 magnetizabilities,
                                 magnetizations,
                                 heat_capacities, alpha=0, beta=1/8, gamma=7/4, nu=1)

In [ ]:
analysis.chi_squared_data_collapse(magnetizabilities, critical_temperature, critical_temperature_error)

In [ ]:
files = ["20160513_150716_ising_wolff_10_65536_[2.269-2.269]_0.1.pickle",
"20160513_150831_ising_wolff_20_65536_[2.269-2.269]_0.1.pickle",
"20160513_151121_ising_wolff_40_65536_[2.269-2.269]_0.1.pickle",
"20160513_151640_ising_wolff_60_65536_[2.269-2.269]_0.1.pickle",
"20160513_152459_ising_wolff_80_65536_[2.269-2.269]_0.1.pickle"]

(critical_temperature,
 critical_temperature_error,
 magnetizabilities,
 magnetizations,
 heat_capacities) = analysis.data_analysis(files, show_plots=False)

In [ ]:
beta_over_nu, beta_over_nu_error = analysis.loglog_exponent_finding(magnetizations, 'Magnetization')
alpha_over_nu, alpha_over_nu_error = analysis.loglog_exponent_finding(heat_capacities, 'Heat Capacity')
gamma_over_nu, gamma_over_nu_error = analysis.loglog_exponent_finding(magnetizabilities, 'Magnetizabilities')
print("Beta/Nu = {0} +/- {1}".format(beta_over_nu, beta_over_nu_error))
print("Alpha/Nu = {0} +/- {1}".format(alpha_over_nu, alpha_over_nu_error))
print("Gamma/Nu = {0} +/- {1}".format(gamma_over_nu, gamma_over_nu_error))